In [ ]:
import pandas as pd
import os

input_file = (
    r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LSTMProduct2.csv"
)

column_mapping = {
    "Ekran Boyutu": "Display Size",
    "Ekran Teknolojisi": "Display Technology",
    "Piksel Yoğunluğu": "Pixel Density",
    "Batarya Kapasitesi": "Battery Capacity",
    "Kamera Çözünürlüğü": "Camera Resolution",
    "CPU Üretim Teknolojisi": "CPU Manufacturing",
    "İşletim Sistemi": "Operating System",
    "RAM Kapasitesi": "RAM",
    "Dahili Hafıza": "Internal Storage",
    "Ekran Yenileme Hızı": "Refresh Rate",
    "Model": "Model",
    "5G": "5G",
    "Price": "Price",
}

if not os.path.exists(input_file):
    raise FileNotFoundError(f"Giriş dosyası bulunamadı: {input_file}")

df = pd.read_csv(input_file)

if "Description" not in df.columns:
    raise KeyError("CSV dosyasında 'Description' sütunu bulunamadı.")

parsed_data = []
for _, row in df.iterrows():
    description = str(row.get("Description", ""))
    specs = {}

    for part in description.split(";"):
        if ":" in part:
            key, value = part.split(":", 1)
            key = key.strip()
            value = value.strip()
            english_key = column_mapping.get(key, key)
            specs[english_key] = value

    parsed_data.append(specs)

parsed_df = pd.DataFrame(parsed_data)

df_updated = pd.concat([df, parsed_df], axis=1)

df_updated.to_csv("hello.csv", index=False, encoding="utf-8-sig")
print(
    "✔ Dosya mevcut kolonlar korunarak ve Description bilgileri ayrılarak kaydedildi."
)

✔ Dosya mevcut kolonlar korunarak ve Description bilgileri ayrılarak kaydedildi.


In [ ]:
import pandas as pd
import re

input_file = (
    r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\hello.csv"
)
output_file = "productDetail_updated.csv"

df = pd.read_csv(input_file)

df.fillna(0, inplace=True)


def extract_ipx_number(value):
    if isinstance(value, str):
        match = re.search(r"\d+", value) 
        if match:
            return int(match.group())
    return 0


if "Suya dayanıklılık" in df.columns:
    df["Waterproof"] = df["Suya dayanıklılık"].apply(extract_ipx_number)
if "Toza dayanıklılık" in df.columns:
    df["Dustproof"] = df["Toza dayanıklılık"].apply(extract_ipx_number)

# df.drop(columns=["Suya dayanıklılık", "Toza dayanıklılık"], inplace=True)

# Kaydet
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"{output_file} dosyası başarıyla güncellendi.")

productDetail_updated.csv dosyası başarıyla güncellendi.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\notebooks\product_specs_en.csv"
)

numeric_df = df.select_dtypes(include=['int64', 'float64'])

corr = numeric_df.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Telefon Özellikleri Sayısal Değişkenler Korelasyon Matrisi")
plt.show()

In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\XGBoostData.csv") 

df["ProductName"] = df["Description"].str.extract(r'Model:\s*([^;]+)')

df["Description"] = df["Description"].str.replace(r'Model:\s*[^;]+;\s*', '', regex=True)

df.to_csv("newProductData.csv", index=False,encoding="utf-8-sig")

print(df.head())


In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\Product.csv")

mean_prices = df.groupby("Display Technology")["Price"].mean().sort_values(ascending=False)

price_ratios = mean_prices / mean_prices.min()

result_df = pd.DataFrame({
    "Average Price": mean_prices,
    "Ratio to Highest": price_ratios
})

print(result_df)


In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\Product.csv")
prediction_price = 15000
df["price_diff"] = (df["Price"] - prediction_price).abs()
df = df[df["price_diff"] <= prediction_price * 0.1]  
print(df.head(5))

print(len(df))

In [ ]:
import pandas as pd

df1 = pd.read_csv(r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\product_specs_en.csv')  
df2 = pd.read_csv(r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LSTMPriceHistory.csv')  

df1['Model'] = df1['Model'].str.strip()
df2['Model'] = df2['Model'].str.strip()

model_to_id = df1.set_index('Model')['ProductID'].to_dict()

df2['ProductID'] = df2.apply(
    lambda row: model_to_id.get(row['Model'], row['ProductID']),
    axis=1
)

df2.to_csv(r'C:\Users\EXCALIBUR\Desktop\kass.csv', index=False)

print("✅ CSV2'deki ProductID'ler, Model eşleşmesine göre CSV1'den güncellendi.")


In [ ]:
import pandas as pd

df1 = pd.read_csv(r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\Product.csv')  df2 = pd.read_csv(r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\product_specs_en.csv')  

df1['Model'] = df1['Model'].str.strip()
df2['Model'] = df2['Model'].str.strip()

id_to_model = df1.set_index('ProductID')['Model'].to_dict()

df2['Model'] = df2.apply(
    lambda row: id_to_model.get(row['ProductID'], row['Model']),
    axis=1
)

df2.to_csv(r'C:\Users\EXCALIBUR\Desktop\nesa.csv', index=False)

print("✅ CSV2'deki Model isimleri, ProductID'ye göre CSV1'den güncellendi.")


In [ ]:
import pandas as pd

all_df = pd.read_csv(
    r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\productDetail_updated.csv"
)

ios_df = all_df[all_df["Operating System"] == "iOS"]

try:
    target_df = pd.read_csv(r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\notebooks\ProductnEW.csv")
except FileNotFoundError:
    target_df = pd.DataFrame(columns=all_df.columns)

merged_df = pd.concat([target_df, ios_df], ignore_index=True).drop_duplicates()
merged_df.drop_duplicates()
merged_df.to_csv("Product.csv", index=False, encoding="utf-8-sig")

print("✔ iOS telefonlar başarıyla ios_only.csv dosyasına eklendi.")

✔ iOS telefonlar başarıyla ios_only.csv dosyasına eklendi.


C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_10912\1849587150.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([target_df, ios_df], ignore_index=True).drop_duplicates()


In [13]:
import pandas as pd
df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\updated_phones.csv")
df.drop_duplicates(subset=["ProductID"], keep="first", inplace=True)
df.to_csv("updated.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\Product.csv")

df = df[df["Operating System"] != "iOS"]

df.to_csv("product_without_ios.csv", index=False, encoding="utf-8-sig")

In [4]:
import pandas as pd

df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\productDetail_updated.csv")

def update_model(row):
    if row["Operating System"] == "iOS" and "RAM" not in row["Model"]:
        return f"{row['Model']} {row['RAM']} GB RAM {row['Internal Storage']} GB"
    return row["Model"]

df["Model"] = df.apply(update_model, axis=1)

df.drop_duplicates(subset=["ProductID"], inplace=True)

df.to_csv("updated_phones.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

df_products = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LSTMPriceHistory.csv")
df_currency = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\currencyRates.csv")

df_products["RecordDate"] = pd.to_datetime(df_products["RecordDate"], dayfirst=True, errors="coerce")
df_currency["RecordDate"] = pd.to_datetime(df_currency["RecordDate"], dayfirst=True, errors="coerce")

merged_df = pd.merge(df_products, df_currency, on="RecordDate", how="left")

merged_df["RecordDate"] = merged_df["RecordDate"].dt.strftime("%d.%m.%Y")

print(merged_df.head())

merged_df.to_csv("products_with_currency.csv", index=False)


C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_3600\934758621.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_products = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LSTMPriceHistory.csv")


   ProductID                   ProductName  RecordDate  Price  CurrencyRate
0     966543  Xiaomi 15 Ultra Cep Telefonu  02.03.2025  79999       36.3652
1     966543  Xiaomi 15 Ultra Cep Telefonu  03.03.2025  79999       36.3812
2     966543  Xiaomi 15 Ultra Cep Telefonu  04.03.2025  79999       36.4097
3     966543  Xiaomi 15 Ultra Cep Telefonu  05.03.2025  79999       36.4011
4     966543  Xiaomi 15 Ultra Cep Telefonu  06.03.2025  79999       36.3682
